In [2]:
import pandas.io.sql as sqlio
import pandas as pd
import psycopg2

d=dict(host="iapp179.iil.intel.com", port=5432, dbname='OGRE_LAB', user='art_readonly', password='12345')
conn = psycopg2.connect("host='{host}' port={port} dbname='{dbname}' user={user} password={password}".format(**d))

/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/virtual_env/env/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/virtual_env/env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
# Path to python:
# /nfs/iil/proj/cto/dav/wa/yoniifra/ws/OgreWeb/python3.6/ogre_web_gui.py

q='''select "tableau" , "IsLocal","IsDeleted", "StartTime"  as "ExecuationDate",c."Id", p."Name" as "Project", s."Name" as "Step", "UserName" as "User", "HostName" as "Host" from dbo."ExecSessions" c 
            join dbo."Hosts" h on h."Id" = c."HostId" 
            join dbo."Projects" p on p."Id" = c."ProjectId" 
            join dbo."ProjectSteps" s on s."Id" = c."ProjectStepId" 
            join dbo."Users" u on u."Id" = c."UserId" where  "IsDeleted" = 'false' and p."Name"='BRK_GEN2' and  c."Id"='7aa9ae72-4916-11eb-8cb9-48df3726b230' '''
df= sqlio.read_sql_query(q, conn)
Id=df.Id.iloc[0]
display(df.head())

,tableau,IsLocal,IsDeleted,ExecuationDate,Id,Project,Step,User,Host
0,None,True,False,2020-12-28 16:10:46.066863,7aa9ae72-4916-11eb-8cb9-48df3726b230,BRK_GEN2,TC0,amasalme,JRW10ACTLAB052


In [7]:
q=f'''Select "ExecSessionId", "Title", "Value" from dbo."ExecSessionConstants" where "ExecSessionId"='{Id}' '''
df= sqlio.read_sql_query(q, conn)
display(df.head())

,ExecSessionId,Title,Value
0,7aa9ae72-4916-11eb-8cb9-48df3726b230,itrinit,true
1,7aa9ae72-4916-11eb-8cb9-48df3726b230,itrinittype,
2,7aa9ae72-4916-11eb-8cb9-48df3726b230,runinit,true
3,7aa9ae72-4916-11eb-8cb9-48df3726b230,runinittype,generalconfig
4,7aa9ae72-4916-11eb-8cb9-48df3726b230,registeraccess,jtag


In [8]:
q=f'''select b."ExecSessionId", a."RecordsAssociationId", b."Time", a."Title", a."Value" 
                  from dbo."DataRecords_ParameterRecord" a 
                  join dbo."RecordsAssociations" b on a."RecordsAssociationId" = b."Id" 
                  where b."ExecSessionId"='{Id}' and a."Title" !='rawdata' and b."IsDeleted"='false' '''
df= sqlio.read_sql_query(q, conn)
display(df.head())

,ExecSessionId,RecordsAssociationId,Time,Title,Value
0,7aa9ae72-4916-11eb-8cb9-48df3726b230,c2dde1ce-4919-11eb-b2dd-48df3726b230,2020-12-28 16:34:15.720008,st3_load,-1
1,7aa9ae72-4916-11eb-8cb9-48df3726b230,c2dde1ce-4919-11eb-b2dd-48df3726b230,2020-12-28 16:34:15.720008,st2_load,11
2,7aa9ae72-4916-11eb-8cb9-48df3726b230,c2dde1ce-4919-11eb-b2dd-48df3726b230,2020-12-28 16:34:15.720008,st1_load,11
3,7aa9ae72-4916-11eb-8cb9-48df3726b230,c2dde1ce-4919-11eb-b2dd-48df3726b230,2020-12-28 16:34:15.720008,st3_gain,-1
4,7aa9ae72-4916-11eb-8cb9-48df3726b230,c2dde1ce-4919-11eb-b2dd-48df3726b230,2020-12-28 16:34:15.720008,st2_gain,11


In [13]:
# now let see each test at different row
df.pivot_table(index='RecordsAssociationId', columns='Title', values='Value', aggfunc='first')

Title,channeldescription,chipid,corner,fwbreakpoint,fwexpectedversion,input_amp,input_freq,iter,lane,lpname,...,st1_load,st2_gain,st2_load,st3_gain,st3_load,testname,teststatus,time,tmp,vcc
RecordsAssociationId,,,,,,,,,,,,,,,,,,,,,
018affcc-4917-11eb-b89e-48df3726b230,['--'],27,ntnv,no_bp,no_fw,0.1,3.050e+10,1,2,"{0: 'diglpb0', 1: 'diglpb1', 2: 'exlpb2', 3: '...",...,11,11,11,-1,-1,sine sweep_using_histogram,pass,mon dec 28 16:14:32 2020,no intec attached,no voltage rail configured
02099d34-491a-11eb-836d-48df3726b230,['--'],27,ntnv,no_bp,no_fw,0.1,1.050e+10,1,2,"{0: 'diglpb0', 1: 'diglpb1', 2: 'exlpb2', 3: '...",...,11,24,11,-1,-1,sine sweep_using_histogram,pass,mon dec 28 16:36:01 2020,no intec attached,no voltage rail configured
02db1af6-491c-11eb-b2d8-48df3726b230,['--'],27,ntnv,no_bp,no_fw,0.1,2.850e+10,1,2,"{0: 'diglpb0', 1: 'diglpb1', 2: 'exlpb2', 3: '...",...,11,24,11,-1,-1,sine sweep_using_histogram,pass,mon dec 28 16:50:21 2020,no intec attached,no voltage rail configured
04ce6f1a-4918-11eb-9898-48df3726b230,['--'],27,ntnv,no_bp,no_fw,0.025,2.450e+10,1,2,"{0: 'diglpb0', 1: 'diglpb1', 2: 'exlpb2', 3: '...",...,11,11,11,-1,-1,sine sweep_using_histogram,pass,mon dec 28 16:21:47 2020,no intec attached,no voltage rail configured
070cb6e6-4919-11eb-a075-48df3726b230,['--'],27,ntnv,no_bp,no_fw,0.05,1.850e+10,1,2,"{0: 'diglpb0', 1: 'diglpb1', 2: 'exlpb2', 3: '...",...,11,11,11,-1,-1,sine sweep_using_histogram,pass,mon dec 28 16:29:00 2020,no intec attached,no voltage rail configured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f92d0534-4919-11eb-90ef-48df3726b230,['--'],27,ntnv,no_bp,no_fw,0.1,8.500e+09,1,2,"{0: 'diglpb0', 1: 'diglpb1', 2: 'exlpb2', 3: '...",...,11,24,11,-1,-1,sine sweep_using_histogram,pass,mon dec 28 16:35:46 2020,no intec attached,no voltage rail configured
fa03a330-491b-11eb-8e59-48df3726b230,['--'],27,ntnv,no_bp,no_fw,0.1,2.650e+10,1,2,"{0: 'diglpb0', 1: 'diglpb1', 2: 'exlpb2', 3: '...",...,11,24,11,-1,-1,sine sweep_using_histogram,pass,mon dec 28 16:50:07 2020,no intec attached,no voltage rail configured
fbd5293a-4917-11eb-93d0-48df3726b230,['--'],27,ntnv,no_bp,no_fw,0.025,2.250e+10,1,2,"{0: 'diglpb0', 1: 'diglpb1', 2: 'exlpb2', 3: '...",...,11,11,11,-1,-1,sine sweep_using_histogram,pass,mon dec 28 16:21:32 2020,no intec attached,no voltage rail configured
